# Analysis of correlation between Delta G, Kd and Temperature

In [10]:
import pandas as pd
import yaml
import os
from tqdm import tqdm_notebook
import numpy as np


In [11]:
import warnings
warnings.filterwarnings("ignore")

from abag_affinity.utils.config import read_config, get_resources_paths

In [12]:
config = read_config("../config.yaml")
sabdab_summary_path, sabdab_pdb_path = get_resources_paths(config, "SAbDab")
abdb_summary_path, abdb_pdb_path = get_resources_paths(config, "AbDb")

In [13]:
abdb_pdb_ids = os.listdir(abdb_pdb_path)
abdb_pdb_ids = set([ pdb_id.split("_")[0].lower() for pdb_id in abdb_pdb_ids])
sabdab_df = pd.read_csv(sabdab_summary_path, sep="\t")
sabdab_pdb_ids = set(sabdab_df["pdb"].unique())

overlapping_ids = abdb_pdb_ids.intersection(sabdab_pdb_ids)

In [14]:
overlapping_data = sabdab_df[sabdab_df["pdb"].isin(overlapping_ids)].copy()

In [30]:
gas_constant =  8.31446261815324 # 0.0821

def calc_temp(row):
    kd = row["affinity"]
    delta_g = row["delta_g"] * 4184 # convert kcal to joule 
    return (delta_g / (gas_constant * np.log(1/kd))) - 273.15 # convert kelvin to celsius

def calc_delta_g(row):
    delta_g = -1 * gas_constant * row["temperature_kelvin"] * np.log(1 / row["affinity"])
    return delta_g / 4184 # convert to kcal

In [31]:
temperature_data = overlapping_data[overlapping_data["temperature"].notnull() & overlapping_data["affinity"].notnull() & ~ overlapping_data["temperature"].isin(["Unknown", "SPR", "BLI"])].copy()
temperature_data["temperature_kelvin"] = temperature_data["temperature"].astype(int) + 273.15
temperature_data["affinity"] = temperature_data["affinity"].astype(float)

temperature_data["calc_dg"] = temperature_data.apply(lambda row: calc_delta_g(row), axis=1)

temperature_data[["temperature", "affinity", "calc_dg", "delta_g"]]

,temperature,affinity,calc_dg,delta_g
13,25,1.000000e-10,-13.642470,-13.63
98,25,9.100000e-08,-9.605607,-9.60
99,25,9.100000e-08,-9.605607,-9.60
124,25,6.200000e-09,-11.197205,-11.19
133,25,4.000000e-06,-7.364124,-7.80
...,...,...,...,...
1239,37,1.100000e-10,-14.132812,-14.12
1302,25,1.300000e-09,-12.122776,-12.12
1307,25,4.100000e-08,-10.077988,-10.07
1316,22,8.900000e-09,-10.872509,-10.87


In [32]:
temperature_data["calc_dg"].corr(temperature_data["delta_g"])

0.9995993874336635

In [33]:
np.log(10)

2.302585092994046

In [34]:
np.log(1/10)

-2.3025850929940455